In [1]:
from util.github_helper import get_readme, get_rate_limit_and_wait_time, get_starcount
import pandas as pd
from util.file_helper import write_text_to_file, to_valid_file_name
from urllib.parse import urlparse
import cmarkgfm
from tqdm import tqdm
import time


In [2]:
df = pd.read_csv("data/acl/scraping/papers_with_code.csv")


In [4]:
# In case of error, use this to resume
# df = df.loc[5704:]
repo_file_map = {}
for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
    parsed_url = urlparse(row["code"])
    df.loc[idx, "platform"] = parsed_url.netloc
    df.loc[idx, "repo_dir"] = parsed_url.path[1:]
    if parsed_url.netloc == "github.com":
        if idx % 100 == 0:
            remaining, wait_time = get_rate_limit_and_wait_time()
            if (remaining < 100):
                print(f"Waiting for {wait_time} seconds")
                time.sleep(wait_time)
        readme_content, url = get_readme(parsed_url.path[1:])
        if readme_content:
            df.loc[idx, "stars"] = get_starcount(parsed_url.path[1:])
            df.loc[idx, "readme_url_path"] = url
            filename = to_valid_file_name(row["paper"])
            write_text_to_file(
                readme_content, "data/acl/readme/md/", f"{filename}.md")
            html = cmarkgfm.github_flavored_markdown_to_html(readme_content)
            write_text_to_file(
                html, "data/acl/readme/html/", f"{filename}.html")
            repo_file_map[parsed_url.path[1:]] = filename
df.to_csv("data/acl/scraping/papers_with_code_readme.csv", index=False)
filemap_df = pd.DataFrame(list(repo_file_map.items()),
                          columns=['repo', 'filename'])
filemap_df.to_csv("data/acl/readme/repo_file_map.csv", index=False)


  0%|          | 0/9300 [00:00<?, ?it/s]

Waiting for 788 seconds


 98%|█████████▊| 9100/9300 [2:10:06<02:11,  1.52it/s]  

Waiting for 183 seconds


100%|██████████| 9300/9300 [2:15:34<00:00,  1.14it/s]  


In [ ]:
neurips_df = pd.read_csv("data/paperswithcode/neurips2019_checklist.csv")


In [ ]:
repo_file_map = {}
for idx, row in tqdm(neurips_df.iterrows(), total=neurips_df.shape[0]):
    parsed_url = urlparse(row["url"])
    if parsed_url.netloc == "github.com":
        readme_content, url = get_readme(parsed_url.path[1:])
        if readme_content:
            filename = to_valid_file_name(parsed_url.path[1:])
            write_text_to_file(
                readme_content, "data/paperswithcode/readme/md/", f"{filename}.md")
            html = cmarkgfm.github_flavored_markdown_to_html(readme_content)
            write_text_to_file(
                html, "data/paperswithcode/readme/html/", f"{filename}.html")
            repo_file_map[parsed_url.path[1:]] = filename
filemap_df = pd.DataFrame(list(repo_file_map.items()),
                          columns=['repo', 'filename'])
filemap_df.to_csv("data/paperswithcode/readme/repo_file_map.csv", index=False)
